How to install promisify autorewire socketio notebook?

In [ ]:
var execSync = require('child_process').execSync;try {    require.resolve('bluebird');    require.resolve('socket.io');    require.resolve('socket.io-client');} catch (e) {    execSync('npm install bluebird automock socket.io socket.io-client');}$$.async();var importer = require('../Core');var automock;importer.import('mock all properties and functions using rewire')    .then(r => {        automock = r;        $$.sendResult('automock loaded');    });

How to mock and pass through?



In [1]:
var Promise = require('bluebird');var promisifyMock = (req, dep) => {    let ctx;    ctx = automock.mockValue(req, {        stubCreator: (name) => {            var orig = Promise.promisify(req[name.split('.')[1]], {                multiArgs: true,                context: req            });            //console.log('create stub ' + name);            return function () {                console.log(name + ' (' + arguments[0] + ') in ' + JSON.stringify(dep));                return orig.apply(null, arguments);            };        }    });    return ctx;};promisifyMock;

[Function: promisifyMock]

Create an RPC socker.io server for any module using promisify?



In [2]:
var Observable = require('rxjs/Observable').Observable;var server = require('socket.io');var deps = {}, io = {};var sockifyRequire = function (req, dep) {    if (typeof deps[dep] !== 'undefined') {        return deps[dep];    }    console.log('Requiring ' + dep);    deps[dep] = automock.mockValue(req, {        stubCreator: function (name) {            // don't call original            if (name.split('.').length === 1) {                return req;            }            // var orig = req[name.split('.')[1]];            return function () {                var args = ['resolve', name];                for (var i = 0; i < arguments.length; i++) {                    args[args.length] = arguments[i];                }                return new Observable(function () {                    var handlers = (typeof io.sockets.adapter.rooms[dep] !== 'undefined'                        ? Object.keys(io.sockets.adapter.rooms[dep].sockets)                        : [])                        .map(function (k) {                            return io.sockets.connected[k];                        });                    if (handlers.length == 0) {                        throw new Error('No handlers for ' + name);                    }                    handlers.forEach(function (h) {                        h.emit.apply(h, args);                    });                });            };        },        name: dep    });    console.log(deps);    return deps[dep];};var sockifyServer = function (port) {    io = server.listen(port);    var socketlist = [];    io.sockets.on('connection', function (socket) {        console.log('Client connection');        socketlist.push(socket);        socket.on('result', function (name) {            var args = ['result', name];            for (var i = 1; i < arguments.length; i++) {                args[args.length] = arguments[i];            }            var handlers = (typeof io.sockets.adapter.rooms[name] !== 'undefined'                ? Object.keys(io.sockets.adapter.rooms[name].sockets)                : [])                .map(function (k) {                    return io.sockets.connected[k];                });            handlers.forEach(function (h) {                h.emit.apply(h, args);            });        });        socket.on('call', function (name) {            var args = [];            for (var i = 1; i < arguments.length; i++) {                args[args.length] = arguments[i];            }            var props = name.split('.');            var dep = props[0];            // join the result room            if (!io.sockets.adapter.sids[socket.id][name]) {                socket.join(name);            }            var func = deps[dep];            for (var j = 1; j < props.length; j++) {                func = func[props[j]];            }            var tmpsub;            tmpsub = func.apply(deps[dep], args).subscribe(function () {                tmpsub.unsubscribe();            });        });        socket.on('handle', function (dep, cb) {            console.log('Handler for ' + dep);            if (!io.sockets.adapter.sids[socket.id][dep]) {                socket.join(dep);            }            cb();        });        socket.on('close', function () {            socketlist.splice(socketlist.indexOf(socket), 1);            socketlist.forEach(function (socket) {                socket.destroy();            });            io.server.close();        });    });    return io;};sockifyServer;module.exports = {    sockifyRequire: sockifyRequire,    sockifyServer: sockifyServer};// TODO: output interactive angular component for controlling this server

[Function: sockifyServer]


Mock any module with promisify and socket.io-client?


In [3]:
var client = require('socket.io-client');var sockifyClient = (req, dep, host) => {    let ctx;    ctx = automock.mockValue(req, {        stubCreator: (name) => {            return function () {                var args = ['call', dep, name.split('.')[1]];                for (var i = 0; i < arguments.length; i++) {                    args[args.length] = arguments[i];                }                socket.emit.apply(socket, args);            };        },        name: dep    });    var promises = promisifyMock(ctx, dep);    promises.___close = () => socket.emit('close');    var socket = client.connect(host);    socket.on('connect', function () {        // TODO: socket.emit('handler') service provider        socket.emit('require', dep, function () {        });        socket.on('resolve', function () {        });    });    return promises;};sockifyClient;

[Function: sockifyClient]

Check if sockify server is running?

In [ ]:
var tryReset = false;try {    var io = sockifyServer(8098);} catch (e) {    if (e.code !== 'EADDRINUSE') {        throw e;    } else {        tryReset = true;    }}var fs = sockifyClient(require('fs'), 'fs', 'http://localhost:8098');if (tryReset) {    fs.___close();    var io = sockifyServer(8098);}

In [4]:
$$.async();fs.unlink('test.file')    .catch(e => console.log(e))    .then(() => fs.writeFile('test.file', 'some output'))    .then(() => fs.readFile('test.file'))    .then(r => r.toString())    .then(r => $$.sendResult(r))    .then(() => fs.___close())    .catch(e => $$.sendError(e));

(none).unlink (test.file) in "fs"


(node:39509) DeprecationWarning: sprintf() will be removed in the next major release, use the sprintf-js package instead.


requiring fs
(none).unlink ( test.file ) 
(none).writeFile (test.file) in "fs"
(none).writeFile ( test.file ) 
(none).readFile (test.file) in "fs"
(none).readFile ( test.file ) 


'some output'